In [1]:
%load_ext autoreload
%autoreload 2

## Image Matching Challenge (PhotoTourism)

In [16]:
from imc.utils_imc_benchmark import load_imc_results

scene_set = "test"  # or "val"
df = load_imc_results(f"imc/results/{scene_set}", return_df=True, all_scenes=False, auc_th=5)
print(len(df), "rows loaded")
# For each (method, custom_desc), get the row with the highest all_auc5, keeping all columns
idx = df.groupby(["method", "custom_desc"])["all_auc5"].idxmax()
df_grouped = df.loc[idx].reset_index(drop=True)
# df_grouped = df_grouped.sort_values(by=["all_auc5"], ascending=False)

df_grouped.head(21)  # Show all 11 methods

21 rows loaded


,method,custom_desc,kpts_budget,params,all_rep,all_inliers,all_auc5
0,aliked,,2048kpts,matcher-mnn0.5-ransac-0.75,48.4,438.0,37.3
1,aliked,SANDesc,2048kpts,matcher-mnn0.5-ransac-0.75,48.4,449.0,42.0
2,dedode,,2048kpts,matcher-mnn0.5-ransac-0.75,43.7,310.0,36.1
3,dedode,SANDesc,2048kpts,matcher-mnn0.5-ransac-0.75,43.7,302.0,35.5
4,dedode,SANDescdl,2048kpts,matcher-mnn0.5-ransac-0.75,43.7,323.0,40.5
5,dedode,SANDescds,2048kpts,matcher-mnn0.5-ransac-0.75,43.7,312.0,38.4
6,dedode,dedodeG,2048kpts,matcher-mnn0.5-ransac-0.75,43.8,340.0,39.9
7,disk,,2048kpts,matcher-mnn0.5-ransac-0.75,45.3,446.0,37.5
8,disk,SANDesc,2048kpts,matcher-mnn0.5-ransac-0.75,45.3,451.0,39.5
9,rdd,,2048kpts,matcher-mnn0.5-ransac-0.75,41.6,370.0,41.9


In [ ]:
import math
import matplotlib.pyplot as plt

if scene_set == "val":
    metric = "all_auc5"
    df_plot = df.copy()
    df_plot = df_plot.sort_values(by=["method", "custom_desc", metric], ascending=[True, True, True])

    methods = df_plot['method'].unique()
    n_methods = len(methods)
    n_cols = 3
    n_rows = math.ceil(n_methods / n_cols)

    fig, axes = plt.subplots(n_rows, n_cols, figsize=(6 * n_cols, 4 * n_rows), squeeze=False)
    axes = axes.flatten()

    for i, method in enumerate(methods):
        ax = axes[i]
        df_method = df_plot[df_plot['method'] == method]
        # Group by custom_desc (e.g., '', 'SANDesc', 'G', etc.)
        for custom_desc, df_variant in df_method.groupby('custom_desc'):
            label = method if custom_desc == '' else f"{method} + {custom_desc}"
            x_vals = range(len(df_variant))
            y_vals = df_variant[metric].values
            marker = 'o' if custom_desc == '' else 's'
            linestyle = '-' if custom_desc == '' else '--'
            ax.plot(x_vals, y_vals, label=label, marker=marker, linestyle=linestyle, linewidth=2)
            for x, y, idx in zip(x_vals, y_vals, df_variant.index):
                ax.annotate(str(idx), (x, y), textcoords="offset points", xytext=(0,10), ha='center', fontsize=10)

        ax.set_title(method)
        ax.set_xlabel("")
        ax.set_xticks([])
        ax.set_ylabel(metric)
        ax.legend(loc='best')
        ax.grid(True, alpha=0.3)
        ax.set_ylim(df_plot[metric].min() - 1, df_plot[metric].max() + 5)

    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.suptitle(f"IMC Benchmark - {metric} on {scene_set} set", y=1.02, fontsize=16)
    plt.show()

## HPatches

In [14]:
from utils_benchmark import print_table
from hpatches.utils_hpatches_benchmark import display_hpatches_results

print("="*16+" HPatches Overall Results "+"="*16)
ths = [1,2,3]
df = display_hpatches_results(partition='overall', ths=ths, tostring=False)  # Display results for thresholds 1, 2, and 3

import pandas as pd
# df = pd.DataFrame(df).sort_values(by=[f'HA@{ths[2]}'], ascending=False)
# sort by method and Desc
df = pd.DataFrame(df).sort_values(by=['Method', 'Desc'], ascending=[True, True])
df

================ HPatches Overall Results ================


,Method,Desc,Rep@1,Rep@2,Rep@3,MA@1,MA@2,MA@3,MS@1,MS@2,MS@3,HA@1,HA@2,HA@3
1,aliked,,29.4,48.7,59.0,36.1,59.5,70.9,23.8,38.2,44.9,39.4,66.7,79.1
0,aliked,SANDesc,29.4,48.7,59.0,40.2,67.0,79.5,24.3,38.8,45.0,47.0,70.9,78.9
2,dedode,SANDesc,33.0,46.9,55.6,47.3,68.5,78.1,20.6,28.9,32.4,50.0,71.7,81.1
3,dedode-B,,33.0,46.9,55.6,45.2,66.1,75.8,20.4,28.7,32.4,50.6,73.0,81.1
4,dedode-G,,32.9,47.1,55.8,43.5,64.2,74.0,20.5,29.3,33.2,49.3,69.6,80.0
6,disk,,29.5,45.0,54.3,41.0,63.3,74.1,25.3,37.8,43.5,43.5,65.2,75.6
5,disk,SANDesc,29.5,45.0,54.3,43.6,68.4,80.9,25.3,37.9,43.8,43.7,63.1,75.4
8,ripe,,27.0,42.4,52.0,37.1,58.4,69.8,23.0,35.3,41.5,49.4,71.3,81.1
7,ripe,SANDesc,27.0,42.4,52.0,41.0,65.5,78.7,23.1,35.4,41.5,45.9,70.4,79.6
10,superpoint,,23.1,41.1,53.9,27.5,47.3,59.6,17.3,29.5,36.7,38.5,63.9,76.5


In [15]:
# load json
path = 'hpatches/results/results.json'

import json
with open(path, 'r') as f:
    results = json.load(f)

for t in ths:
    print(f"\n=== Threshold: {t} px ===")
    for k in sorted(list(results.keys())):
        print(f"{k}: {results[k]['overall'][f'homography_accuracy_{t}']['best_ransac_thr']} px")
        print(f"{k}: {results[k]['overall'][f'homography_accuracy_{t}']['mean']} auc \n")
        


=== Threshold: 1 px ===
aliked+SANDesc_2048kpts_20251014_141003: 5.0 px
aliked+SANDesc_2048kpts_20251014_141003: 0.4703703703703704 auc 

aliked_2048kpts_20251014_140035: 5.0 px
aliked_2048kpts_20251014_140035: 0.39444444444444443 auc 

dedode+SANDesc_2048kpts_20251014_124310: 5.0 px
dedode+SANDesc_2048kpts_20251014_124310: 0.5 auc 

dedode-B_2048kpts_20251014_124449: 4.0 px
dedode-B_2048kpts_20251014_124449: 0.5055555555555555 auc 

dedode-G_2048kpts_20251014_124710: 3.0 px
dedode-G_2048kpts_20251014_124710: 0.4925925925925926 auc 

disk+SANDesc_2048kpts_20251014_140838: 0.5 px
disk+SANDesc_2048kpts_20251014_140838: 0.4368029739776952 auc 

disk_2048kpts_20251014_135545: 5.0 px
disk_2048kpts_20251014_135545: 0.4351851851851852 auc 

ripe+SANDesc_2048kpts_20251014_141121: 5.0 px
ripe+SANDesc_2048kpts_20251014_141121: 0.45925925925925926 auc 

ripe_2048kpts_20251014_140159: 5.0 px
ripe_2048kpts_20251014_140159: 0.49444444444444446 auc 

superpoint+SANDesc_2048kpts_20251014_140643: 5.0 

## Megadepth-1500

In [1]:
import os
import pandas as pd

which_megadepth = "megadepth1500"  # or "megadepth_view" or "megadepth_air2ground"
results = f'{which_megadepth}/results/results.json'
if not os.path.exists(results):
    raise FileNotFoundError(f"Results file not found: {results}. First run the benchmark!")

df = pd.read_json(results).T
pairs = f"{int(df['total_pairs'].unique()[0]):,}"
df['inliers'] = round(df['inliers']).astype(int)
df[f'unregistered/{pairs}'] = df['unregistered_pairs'].astype(int)
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['budget'] = df.index.str.split(' ').str[2]
df['note'] = df.index.str.split(' ').str[3]
df['timestamp'] = df.index.str.split(' ').str[-1]

# filter
df = df[
    # df['model'].str.contains('dedode') | df['model'].str.contains('rdd')
    # & ~df['note'].str.contains('None')
    df['params'].str.contains('ratio_test_1.0_th_0.75')
] 

df = df[['model', 'budget', 'inliers', f'unregistered/{pairs}', 'auc_5', 'auc_10', 'timestamp', 'params']]
# df.sort_values(by=['budget', 'model'], inplace=True, ascending=[True, True])

#  group by model, sort by auc5, keep first
df = df.sort_values(by=['model', 'auc_5'], ascending=[True, False])
# df = df.groupby('model').first().reset_index()
# df = df.sort_values(by=['auc_5'], ascending=False)
df.index = range(1, 1+len(df.index))
df.round(1)

,model,budget,inliers,"unregistered/1,500",auc_5,auc_10,timestamp,params
1,aliked,2048,457,4,41.8,57.7,20251002_115803,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
2,aliked+SANDesc,2048,475,12,42.0,58.6,20251002_115930,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
3,dedode,2048,271,7,44.1,61.1,20251002_121602,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
4,dedode+SANDesc,2048,274,8,46.2,62.2,20251002_121947,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
5,dedode+SANDescDl,2048,294,12,47.8,64.8,20251002_115429,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
6,dedode+SANDescDs,2048,282,15,45.3,62.8,20251002_115136,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
7,dedode-G,2048,299,8,46.4,64.1,20251002_122234,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
8,disk,2048,456,5,35.5,52.3,20251002_120125,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
9,disk+SANDesc,2048,477,9,38.0,54.9,20251002_120255,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...
10,rdd,2048,360,15,46.7,63.6,20251002_114139,min_score_0.5_ratio_test_1.0_th_0.75_mnn_scale...


## Graz High Resolution

In [ ]:
scale_factors = {'scale_1.0': '4K', 'scale_1.5': 'QHD', 'scale_2.0': 'FHD'}

results = 'graz_high_res/results/results_paper.json'
if not os.path.exists(results):
    raise FileNotFoundError(f"Results file not found: {results}. First run the benchmark!")

df = pd.read_json(results).T.round(3)
df['inlier'] = round(df['inlier']).astype(int)
df['auc_5'] *= 100
df['auc_10'] *= 100
df['model'] = df.index.str.split(' ').str[0]
df['params'] = df.index.str.split(' ').str[1]
df['resolution'] = [scale_factors[s] for s in df.index.str.split(' ').str[2]]
df['budget'] = df.index.str.split(' ').str[3]
df['timestamp'] = df.index.str.split(' ').str[-1]

# filter
# df = df[df['resolution'].str.contains('FHD')] # comment to display all resolutions

df_auc = df[['model', 'budget', 'resolution', 'inlier', 'auc_5', 'auc_10', 'timestamp']].copy()
df_auc.sort_values(by=['resolution', 'model'], inplace=True, ascending=[False, True])
df_auc.index = range(1, 1+len(df_auc.index))
df_auc # there should be 11 elements for FHD, and 9 for QHD and 4K

### Scenes breakdown

Run this after running the benchmark, otherwise no statistics are available.

In [ ]:
import json
from benchmark_utils import pose_auc

with open('graz_high_res/results/results_paper.json', 'r') as f:
    data = json.load(f)
    
keys = list(data.keys())
auc_th = 5
scale = 1 # 1, 1.5, 2

breakdown, total = {}, {}
for key in keys:
    parts = key.split(' ')
    key = f'{parts[0]}_stats_scale_{float(scale)}_{parts[4]}_{parts[6]}'
    model = parts[0]

    try:
        os.path.exists(f'graz_high_res/stats/{key}.csv')
        df = pd.read_csv(f'graz_high_res/stats/{key}.csv')
    except FileNotFoundError:
        continue

    scenes = df.scene_name.unique()
    total[model] = round(pose_auc(df.e_pose, [auc_th])[0] * 100,1)

    for scene in scenes:
        scene_df = df[df.scene_name == scene]
        if scene not in breakdown:
            breakdown[scene] = {}
        breakdown[scene][model] = round(pose_auc(scene_df.e_pose, [auc_th])[0] * 100,1)

breakdown['total'] = total

In [ ]:
df_breakdown = pd.DataFrame.from_dict(breakdown, orient='index').T
df_breakdown